In [1]:
import os
import pandas as pd
import numpy as np
from importlib import reload

import data_wrangle
import spectral
import analysis
import __utils__

from prettytable import PrettyTable as pt

#From PCA
from sklearn.decomposition import PCA
from sklearn import decomposition
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import scipy.signal as ss
import pybaselines.spline as py

from math import ceil
from collections import Counter

# import plotly.io as pio
# pio.renderers.default = "browser"
# pio.renderers.default = "png"

# Initalize Object and set constants

In [4]:
path='/mnt/c/Users/16162/Desktop/Brain.db'
table='data'

## Look at the Table

In [131]:
reload(data_wrangle)
db=data_wrangle.DataTable(path,table)
db.summary('date')

db.__del__()

all
pirate
Currently the dataset is accessing 539 samples from the database.
+-----------------------------------------------+
|Summaries of the data in the selected columns: | 
+------------------------+
|          date          |
+------------+-----------+
|   Value    | Frequency |
+------------+-----------+
| 2022-08-15 |     11    |
| 2022-10-12 |     22    |
| 2022-11-03 |     15    |
| 2022-11-10 |     34    |
| 2023-01-10 |     34    |
| 2023-03-01 |     30    |
| 2023-03-03 |     24    |
| 2023-03-16 |     21    |
| 2023-03-24 |     48    |
| 2023-03-27 |     54    |
| 2023-03-29 |     90    |
| 2023-03-30 |    156    |
+------------+-----------+


# Grab data
---

+ Can remove spectra by id:  
`db.deab_list.append(#)`   
+ Can set conditions:  
    + `db.condition='WHERE col IS NOT NULL'`  
    + `db.condition='WHERE col IS value'`  
    + `db.condition='WHERE col1 IS NOT NULL AND col2 = value'`
    + `db.condition='WHERE strftime(\'%Y\',date)=\'2023\' AND general_loc in (\'substantia nigra\',\'cortex\',\'midbrain\')'`  
    + `db.condition="WHERE date LIKE '2023-%'"# AND location IN ('cortex')"`
+ As currently write, `raman_shifts()` should be executed twice to automatically remove problematic Raman Shifts

In [182]:
reload(__utils__)
reload(data_wrangle)
reload(analysis)
db=data_wrangle.DataTable(path, table)

db.condition = 'WHERE location IS NOT NULL'

data=db.dataset(summary=False)

rs=data.index
id=data.columns
names=db.label_dict('date',include_col_name=False)

var=analysis.multivar(data.T,rs,names)

pc,a,b=var.pca_run()
var.pca_fig('PC 1', 'PC 2')



pxfig note: add a color blind mode option here


In [154]:
reload(__utils__)
reload(data_wrangle)
db=data_wrangle.DataTable(path, table)

db.condition = 'WHERE location IS NOT NULL'

db.summary()

All IDs: 539
Selected IDs: 408
Condition: WHERE location IS NOT NULL


In [200]:
df=pd.DataFrame(data={
    'A':['NaN',2,1,3,4,9,6,7,10],
    'B':[0,3,2,3,5,6,7,'NaN','NaN']})

res=pd.DataFrame(data={
    'A':[2,1,3,4,9,6],
    'B':[3,2,3,5,6,7]},
    index=[1,2,3,4,5,6])



,A,B
1,2,3
2,1,2
3,3,3
4,4,5
5,9,6
6,6,7


In [162]:
reload(__utils__)
lst=[list(s) for s in summary]
mylist=[]
res=[]
for l in lst:
    new=__utils__.modify_duplicates(l)
    stmt=f'len original: {len(l)}\tlen new: {len(new)}\nlen set original: {len(list(set(l)))}\tlen set new: {len(list(set(new)))}\n'
    print(stmt)

len original: 1100	len new: 1100
len set original: 893	len set new: 1100

len original: 1100	len new: 1100
len set original: 891	len set new: 1100

len original: 1340	len new: 1340
len set original: 1059	len set new: 1340

len original: 1340	len new: 1340
len set original: 1063	len set new: 1340

len original: 1100	len new: 1100
len set original: 889	len set new: 1100

len original: 1100	len new: 1100
len set original: 889	len set new: 1100



In [155]:
reload(__utils__)
reload(data_wrangle)
db=data_wrangle.DataTable(path, table)

# #1) Add search conditiom *optional*
# db.condition = 'WHERE location IS NOT NULL'

#2) Get X and Y data

x=db.raman_shifts()
# ys=db.intens()
# x=db.raman_shifts()
# ys=db.intens()


#3) Generate labeling dictionaries
# namedic=db.label_dict(cols=['location'])
# date=db.label_dict(cols=['date'])

# __utils__.dict_summary(namedic)

# View spectra

In [ ]:
#3) Make color dictionary
for item in set(namedic.values()):
    print(f'\'{item}\':\'#\',',)

#4) Make set dictionary

In [4]:
color_dic={
'19 uM As3, 1 mM As5':'#8200ff',
'0 As3, 150 uM As5':'#6600ff',
'0 As3, 1 mM As5':'#250094',
'19 uM As3, 0 As5':'#ff0061',
'0 As3, 0 As5':'grey',
'19 uM As3, 150 uM As5':'#da00c6',
'50 uM As3, 0 As5':'#b40000',
'50 uM As3, 1 mM As5':'#a800e7',
'50 uM As3, 150 uM As5':'#ff00ae',
}

color_dic={
'0 As3, 1 mM As5':'#250094',
'0 As3, 150 uM As5':'#00eaff',
'50 uM As3, 150 uM As5':'#824c96',
'19 uM As3, 150 uM As5':'#433466',
'50 uM As3, 1 mM As5':'#ff7a00',
'19 uM As3, 1 mM As5':'#ed733f',
'19 uM As3, 0 As5':'#ff7a00',
'50 uM As3, 0 As5':'#ff0000',
'0 As3, 0 As5':'grey',}

color_dic={
'0 As3, 1 mM As5':'#2abb1d',
'0 As3, 150 uM As5':'#4ded30',

'50 uM As3, 150 uM As5':'#d1022f',
'19 uM As3, 150 uM As5':'#9f0261',
'50 uM As3, 1 mM As5':'#7c0284',
'19 uM As3, 1 mM As5':'#5102af',

'19 uM As3, 0 As5':'#e05a00',
'50 uM As3, 0 As5':'#a34100',

'0 As3, 0 As5':'grey',}

set_dic={
  'Mix':['50 uM As3, 1 mM As5', '19 uM As3, 1 mM As5', '19 uM As3, 150 uM As5', '50 uM As3, 150 uM As5'],
  'As3':['50 uM As3, 0 As5','19 uM As3, 0 As5'],
  'As5':['0 As3, 1 mM As5','0 As3, 150 uM As5'],
  'Blank':['0 As3, 0 As5']
 }

In [20]:
reload(spectral)

spec=spectral.Spectral(ys,x,namedic,date)
# spec.trunc_before_x=314
# spec.trunc_after_x=1663

spec.single_plot()

# spec.set_plot(peak_list=mix)

# fig=spec.set_plot(move_y_title=0.08)
# fig.show()
# mix=[411,445,496,563,670,700,765,810,850,951,1074,1330,1379,1590]
# as3=[371,409,500,566,761,838,961,994,1021,1181,1223,1253,1374,1436,1537,1590,1642]

# fig=spec.peak_label(fig,mix,row=1) #1.05
# fig=spec.peak_label(fig,mix,row=2) #0.785
# fig=spec.peak_label(fig,as3,row=3) #0.47
# spec.peak_label(fig,as3,row=4) #0.16

In [ ]:
for i in fig['data']:
    print(color_dic[i['name']])

In [34]:
print(ys.shape)
print(x.shape)
print(x)
print(db.dead_list)

(408, 1340)
(0,)
Series([], dtype: float64)
[117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 318, 319, 320, 321, 322, 

# PCA

In [22]:
reload(analysis)
var=analysis.multivar(ys,x,namedic, trunc_before_x=650)

# pc,a,b=var.pca_run()
var.pca_fig('PC 1', 'PC 2')
# var.pca_fig('PC 1', 'PC 3')
# # var.pca_fig('PC 2', 'PC 3')
# var.pca_3d().show()
# var.loo()




ValueError: Length mismatch: Expected axis has 1340 elements, new values have 1100 elements

In [ ]:
var.loo()

In [ ]:
var.pca_fig('PC 1', 'PC 2')

In [ ]:
ls=var.loadings('1','2', '3')
for l in ls:
    l.show()


# Extras!

## Update Database Find and Replace

In [ ]:
reload(data_wrangle)
db=data_wrangle.DataTable('/mnt/c/Users/16162/Desktop/Brain.db','sheep')
db.condition='WHERE sheep IS 2'
db.data_info('nanotag') #Add 'condition' to find_replace
# db.find_replace('BoooTh', 'Booo', 'nanotag')

## Making Figure - old code

In [ ]:
reload(__utils__)
#This should go into spectral
def truncate(x_data, y_data, before_target=None, after_target=None):
    '''targets need to be intergers for the Raman Shift values.
    x_data = Raman Shifts as a series or list
    y_data = df where each row is a sample
    If no only 1 target is given, this should still work'''#***
    #Find indexes for truncating
    if before_target==None:
        before_idx=0
    else:
        before_idx=__utils__.find_index(x_data, before_target)
    if after_target==None:
        after_idx=len(x_data)
    else:
        after_idx=__utils__.find_index(x_data, after_target)
    print(after_idx,'FIX return error if NONE')
    res_x=x_data[before_idx:after_idx]
    res_ys=y_data.iloc[:,before_idx:after_idx]
    t1=go.Scatter(x=x_data, y=y_data.iloc[1], name='before')
    t2=go.Scatter(x=res_x, y=res_ys.iloc[1], name='after')
    traces=[t1,t2]
    fig=go.Figure(traces)
    fig.update_layout(title='Visualize Data Truncation')
    fig.show()

    return res_x, res_ys

# x,y=truncate(x,ys,500,1080)

## Find and remove NaN values

In [ ]:
#Find NaN values
nan_cols=ys.isna().any() #Find columns with NaN values
nan_rows=ys.loc[:,nan_cols].isna().any(axis=1) #Find the rws with NaN values (in columns with NaN values)
nan_rows.index #Get indexes of rows (= sample ID)

#Drop rows with more than 5 NaN values
ys=ys.dropna(thresh=ys.shape[1]-5)

#Map namedic to sample ID
tag=ys.index.map(lambda x: namedic[x][0])

## Something to do with dataframe manipulation

In [ ]:
df=pd.DataFrame([[1,2,3,4,5],[2,3,4,5,6],[1,2,3,4,5],[5,6,7,8,9],[1,2,3,4,5],[5,6,7,8,9]])
print(df)
uniques=set(map(tuple,df.values))
uniques=list(uniques)
for u in uniques:
    mask=np.all(df==u, axis=1)
    print(mask)
print(df[mask])
print(df)
